In [11]:
import pandas as pd
import sqlite3

# Tạo bảng student
students = pd.DataFrame({
    'student_id': [1, 2, 3, 4, 5],
    'name': ['A', 'B', 'C', 'D', 'E'],
    'course_id': [101, 102, 103, None, 105],
    'score': [9.5, 7.8, 5.4, 8.2, 6.7]
})

# Tạo bảng course
courses = pd.DataFrame({
    'course_id': [101, 102, 103, 104, 105],
    'course_name': ['Math', 'Physics', 'Chemistry', 'Biology', 'English']
})

print("Bảng student:")
print(df_student)
print("\nBảng course:")
print(df_course)


Bảng student:
   student_id     name  course_id
0           1    Alice      101.0
1           2      Bob      102.0
2           3  Charlie      103.0
3           4    David        NaN
4           5      Eve      101.0

Bảng course:
   course_id course_name
0        101        Math
1        102     Physics
2        103   Chemistry
3        104     Biology


#Câu 1

##Sử dụng tích Descartes

In [12]:
df_cross = df_student.merge(df_course, how='cross')
print("\nCROSS JOIN:")
print(df_cross)



CROSS JOIN:
    student_id     name  course_id_x  course_id_y course_name
0            1    Alice        101.0          101        Math
1            1    Alice        101.0          102     Physics
2            1    Alice        101.0          103   Chemistry
3            1    Alice        101.0          104     Biology
4            2      Bob        102.0          101        Math
5            2      Bob        102.0          102     Physics
6            2      Bob        102.0          103   Chemistry
7            2      Bob        102.0          104     Biology
8            3  Charlie        103.0          101        Math
9            3  Charlie        103.0          102     Physics
10           3  Charlie        103.0          103   Chemistry
11           3  Charlie        103.0          104     Biology
12           4    David          NaN          101        Math
13           4    David          NaN          102     Physics
14           4    David          NaN          103   Chemi

##Sử dụng JOIN

In [13]:
#INNER JOIN
df_inner = df_student.merge(df_course, on='course_id', how='inner')
print("\nINNER JOIN:")
print(df_inner)
#LEFT JOIN
df_left = df_student.merge(df_course, on='course_id', how='left')
print("\nLEFT JOIN:")
print(df_left)
#RIGHT JOIN
df_right = df_student.merge(df_course, on='course_id', how='right')
print("\nRIGHT JOIN:")
print(df_right)
#FULL OUTER JOIN
df_outer = df_student.merge(df_course, on='course_id', how='outer')
print("\nFULL OUTER JOIN:")
print(df_outer)


INNER JOIN:
   student_id     name  course_id course_name
0           1    Alice      101.0        Math
1           2      Bob      102.0     Physics
2           3  Charlie      103.0   Chemistry
3           5      Eve      101.0        Math

LEFT JOIN:
   student_id     name  course_id course_name
0           1    Alice      101.0        Math
1           2      Bob      102.0     Physics
2           3  Charlie      103.0   Chemistry
3           4    David        NaN         NaN
4           5      Eve      101.0        Math

RIGHT JOIN:
   student_id     name  course_id course_name
0         1.0    Alice      101.0        Math
1         5.0      Eve      101.0        Math
2         2.0      Bob      102.0     Physics
3         3.0  Charlie      103.0   Chemistry
4         NaN      NaN      104.0     Biology

FULL OUTER JOIN:
   student_id     name  course_id course_name
0         1.0    Alice      101.0        Math
1         5.0      Eve      101.0        Math
2         2.0      Bob  

#Câu 2

In [17]:
# Cập nhật course_id bị thiếu bằng giá trị hợp lệ từ bảng course
cursor.execute("""
UPDATE student
SET course_id = (SELECT MIN(course_id) FROM course)
WHERE course_id IS NULL;
""")
# Xóa những sinh viên có course_id không hợp lệ
cursor.execute("""
DELETE FROM student
WHERE course_id NOT IN (SELECT course_id FROM course);
""")
conn.commit()
#Tổng số sinh viên và điểm trung bình từng lớp
cursor.execute("""
SELECT course_id, COUNT(student_id) AS total_students, AVG(score) AS avg_score
FROM student
GROUP BY course_id;
""")
print("\nTổng số sinh viên và điểm trung bình của từng lớp:")
for row in cursor.fetchall():
    print(row)

#Tổng số sinh viên và điểm trung bình từng môn học
cursor.execute("""
SELECT c.course_name, COUNT(s.student_id) AS total_students, AVG(s.score) AS avg_score
FROM student s
JOIN course c ON s.course_id = c.course_id
GROUP BY c.course_name;
""")
print("\nTổng số sinh viên và điểm trung bình của từng môn học:")
for row in cursor.fetchall():
    print(row)

#Phân loại thi đua theo số điểm từng môn học
cursor.execute("""
SELECT c.course_name,
       COUNT(s.student_id) AS total_students,
       AVG(s.score) AS avg_score,
       CASE
           WHEN AVG(s.score) >= 9.0 THEN 'Xuất sắc'
           WHEN AVG(s.score) BETWEEN 6.0 AND 8.9 THEN 'Tốt'
           ELSE 'Kém'
       END AS classification
FROM student s
JOIN course c ON s.course_id = c.course_id
GROUP BY c.course_name;
""")
print("\nPhân loại thi đua theo số điểm:")
for row in cursor.fetchall():
    print(row)



Tổng số sinh viên và điểm trung bình của từng lớp:
(12, 4, 7.2)
(34, 2, 9.2)

Tổng số sinh viên và điểm trung bình của từng môn học:
('Giai tich', 4, 7.2)
('Thong ke', 2, 9.2)

Phân loại thi đua theo số điểm:
('Giai tich', 4, 7.2, 'Tốt')
('Thong ke', 2, 9.2, 'Xuất sắc')


#Câu 3

In [18]:
#Xếp hạng sinh viên
##Theo điểm số
cursor.execute("""
SELECT student_id, name, score,
       RANK() OVER (ORDER BY score DESC) AS rank
FROM student;
""")
print("\nXếp hạng sinh viên theo điểm số:")
for row in cursor.fetchall():
    print(row)

##Điểm số theo lớp học
cursor.execute("""
SELECT student_id, name, course_id, score,
       RANK() OVER (PARTITION BY course_id ORDER BY score DESC) AS rank
FROM student;
""")
print("\nXếp hạng sinh viên theo điểm số trong từng lớp học:")
for row in cursor.fetchall():
    print(row)

##Điểm số theo mã môn học
cursor.execute("""
SELECT s.student_id, s.name, c.course_name, s.score,
       RANK() OVER (PARTITION BY s.course_id ORDER BY s.score DESC) AS rank
FROM student s
JOIN course c ON s.course_id = c.course_id;
""")
print("\nXếp hạng sinh viên theo điểm số trong từng môn học:")
for row in cursor.fetchall():
    print(row)



Xếp hạng sinh viên theo điểm số:
(2, 'Tran Thi Lan', 9.2, 1)
(7, 'Bui Tien Dung', 9.2, 1)
(3, 'Pham Van Nam', 7.9, 3)
(9, 'Duong Huu Phuc', 7.2, 4)
(10, 'Cao Thi Hanh', 7.0, 5)
(1, 'Nguyen Minh Hoang', 6.7, 6)

Xếp hạng sinh viên theo điểm số trong từng lớp học:
(3, 'Pham Van Nam', 12, 7.9, 1)
(9, 'Duong Huu Phuc', 12, 7.2, 2)
(10, 'Cao Thi Hanh', 12, 7.0, 3)
(1, 'Nguyen Minh Hoang', 12, 6.7, 4)
(2, 'Tran Thi Lan', 34, 9.2, 1)
(7, 'Bui Tien Dung', 34, 9.2, 1)

Xếp hạng sinh viên theo điểm số trong từng môn học:
(3, 'Pham Van Nam', 'Giai tich', 7.9, 1)
(9, 'Duong Huu Phuc', 'Giai tich', 7.2, 2)
(10, 'Cao Thi Hanh', 'Giai tich', 7.0, 3)
(1, 'Nguyen Minh Hoang', 'Giai tich', 6.7, 4)
(2, 'Tran Thi Lan', 'Thong ke', 9.2, 1)
(7, 'Bui Tien Dung', 'Thong ke', 9.2, 1)


#Câu 4

In [19]:
# CÂU 4
import sqlite3
import datetime
import pandas as pd

# Kết nối CSDL SQLite
conn = sqlite3.connect("university.db")
cursor = conn.cursor()

# Kiểm tra xem cột graduation_date đã tồn tại chưa
cursor.execute("PRAGMA table_info(student)")
columns = [col[1] for col in cursor.fetchall()]

if "graduation_date" not in columns:
    cursor.execute("ALTER TABLE student ADD COLUMN graduation_date TEXT")  # Dùng TEXT thay vì DATETIME
    conn.commit()
    print("Đã thêm cột graduation_date.")
else:
    print("Cột graduation_date đã tồn tại, bỏ qua ALTER TABLE.")

# Cập nhật ngày tốt nghiệp (hiện tại + số tháng tương ứng với điểm số)
for row in cursor.execute("SELECT student_id, score FROM student"):
    student_id, score = row
    grad_date = (datetime.datetime.now() + datetime.timedelta(days=int(score * 30))).strftime('%Y-%m-%d')  # Chuyển sang dạng chuỗi
    cursor.execute("UPDATE student SET graduation_date = ? WHERE student_id = ?", (grad_date, student_id))

conn.commit()

# Kiểm tra kết quả
df_grad = pd.read_sql_query("SELECT student_id, name, graduation_date FROM student", conn)
print(df_grad)

# Đóng kết nối
conn.close()

Đã thêm cột graduation_date.
   student_id               name graduation_date
0           1  Nguyen Minh Hoang      2025-09-29
1           2       Tran Thi Lan            None
2           3       Pham Van Nam            None
3           7      Bui Tien Dung            None
4           9     Duong Huu Phuc            None
5          10       Cao Thi Hanh            None
